# Libraries

In [60]:
import pandas as pd
import seaborn as sns
import regex as re

import spacy

import tqdm

import numpy as np

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import pyLDAvis
import pyLDAvis.gensim_models as gensim_models
import pyLDAvis.gensim_models as gensimvis


import gensim
# from gensim.models import wrappers
# from gensim.models.wrappers import LdaMallet
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD


import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer


from pprint import pprint

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning)

# Importing -- Fixing columns

In [2]:
full = pd.read_csv('Data/Data-Cleaned/238k-Uncleaned')

In [3]:
# Renaming tweets column, dropping unnamed column, making tweets strings

full['tweets'] = full['0']
full = full.drop(columns = '0')
full.set_index('Unnamed: 0')
# full['tweets'] = full['tweets'].astype('string')
# full.dropna(inplace = True)

,tweets
Unnamed: 0,
1527025892458999814,@testcranker @ITGuy1959 My solar is 100% off g...
1527025892031086593,Boycott #Tesla #TeslaStock #ElonsProblems Vote...
1527025884690993153,@dinodlz @cmclymer Tesla never would have surv...
1527025878974271489,@MarketRebels @elwalvador It would be funny if...
1527025852008976384,@RiceAndrew Right - it's just his nonsense pos...
...,...
1525643350828777473,"@ValueAnalyst1 Yes, beg - borrow and steal to ..."
1525642529567571969,@iamDCinvestor Owh Have to rethink about my th...
1525639115051515909,"@5050isaloss @cardano_whale Overused example, ..."


# Code for Sentiment ---- ---- NOT NEEDED FOR MODELING

In [8]:
# CODE WORKS --- JUST USE IT LATER 

# Adding Sentiment
sia = SentimentIntensityAnalyzer()
comp_dic = {}
for i, tweet in full['tweets'].iteritems():
    comp_dic[i] = sia.polarity_scores(tweet)['compound']

comp_scores = pd.Series(comp_dic, name = 'sentiment')
df_sent = full.merge(comp_scores, left_index = True, right_index = True)
df = df_sent.set_index('Unnamed: 0')

# Preprocessing ---- will need to test different preprocessing later

In [ ]:
"""
The goal here is to get to a final model with the full data and an iteration of cleaned preprocessing
Clean this notebook as much as possible
make a copy of the notebook-- save the original and MUST get bi and trigrams to work


"""

In [9]:
def Series_Preprocessor(tweet):
    stopWords = stopwords.words('english')
    stopWords.extend(['tesla', 'c,', 'x', 't', 'p', 'amp'])
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()

        # helper function to change nltk's part of speech tagging to a wordnet format.
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None 
    
#   Lowercase all 
    tweet = tweet.lower()
    
#   Removing @ handles, links-- strip whitespace breaks and tabs
    tweet = re.sub(r"@\w+|http\S+", "", tweet).strip().replace("\r", "").replace("\n", "").replace("\t", "")
    
#   Tokenize,make list of words, removing punctuation and stopwords
    tweet = [x for x in word_tokenize(tweet) if ((x.isalpha()) & (x not in stopWords)) ]
    
#   Map part of speech tags to words and use words/tags to lemmatize accuratley
    tweet = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(tweet))) 
    tweet = " ".join([wnl.lemmatize(x[0], x[1]) for x in tweet if x[1] is not None])

    return tweet 

In [10]:
df = full['tweets'].apply(Series_Preprocessor)

## Need to make Bi and Tri Grams ---- Do this below

#### ============================================================================

#### ============================================================================

## Will need to change entire preprocessing step to make bigrams and trigrams -- 
## Therefore the following code will be using the traw imported data

In [11]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237659 entries, 0 to 237658
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  237659 non-null  int64 
 1   tweets      237659 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [12]:
# remove punctuation
full['preprep'] = full['tweets'].map(lambda x: re.sub('[,\.!?]', '', x))
# Lowercase
full['preprep'] = full['preprep'].map(lambda x: x.lower())
# #   Removing @ handles, links-- strip whitespace breaks and tabs
full['preprep'] = full['preprep'].map(lambda x: re.sub(r"@\w+|http\S+", "", x).strip().replace("\r", "").replace("\n", "").replace("\t", ""))

In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = full.preprep.values.tolist()
data_words = list(sent_to_words(data))

In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=3, threshold=75) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=60)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [15]:
stop_words = stopwords.words('english')
stop_words.extend(['tesla', 'c,', 'x', 't', 'p', 'amp', 'car', 'go'])
wnl = WordNetLemmatizer()

In [16]:
# Functions to for preprocessing for bigrams and trigrams texts, all tested and edited all should be good 

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words]for doc in texts]

def make_bigrams(texts):
    return bigram_mod[texts]

def make_trigrams(texts):
    return trigram_mod[bigram_mod[texts]]

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None 
    
def lemmatize_texts(tweet):
     for i in tweet:
        word = (map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(tweet))) 
        word_lem = " ".join([wnl.lemmatize(x[0], x[1]) for x in word if x[1] is not None])
        return word_lem

In [17]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# lemmatize
data_lemmatized = [lemmatize_texts(tweet) for tweet in data_words_bigrams]

In [18]:
#proved that Nones are introduced during the lemmatization step -- there are no nones before this step
# This will need to be fixed later
lem_noNone = [x for x in data_lemmatized if x is not None]

## Find missing values after making min tweet length 4 words maybe =============================

In [54]:
# testing enumerate with above lemmatization function
# tester = [(i,x) for (i,x) in enumerate(data_lemmatized) if x is not None]

def find_missing(lst):
    return [x for x in range(lst[0], lst[-1]+1) 
                               if x not in lst] 
lst = [tup[0] for tup in tester]
ind = (find_missing(lst))
ind

In [ ]:
"""
What was found here:

the numbers in 'ind' correspond perfectly with the indices in full.iloc[i]
if i were to drop these values prreprocessing the notebook would be a weird loop in whcih i can only drop values from 
the original df once ive found them after the preprocessing.... 


"""

In [53]:
# for i in ind: 
#     print(i, full['preprep'][i])

In [55]:
# for i in ind: 
#     print(i, full.tweets.iloc[i])

587 @DriveTeslaca @tesla_adri @teslascope Nor here.
676 @Tesla_tunes Not all
825 @Sum46193023 @mary_led_what @SawyerMerritt @Tesla They are.
1631 @facts_tesla @AssaadRazzouk @zshahan3 @JohnnaCrider1 @EvaFoxU But but … https://t.co/IbnM6AP8Zz
1643 @MUTheadTy @ShockerSports13 @Jonathan_Jones2 Not if you have a Tesla 🤷🏻‍♂️
1711 @sidmas88 @Tesla for what?
1811 Couldn’t not

 #Tesla https://t.co/vW8WBJxB4g
1925 @AmirSabzinezhad @Tesla There https://t.co/CKHSmxWVpF
2152 Is it Tesla?? https://t.co/ySVjwiEZSj
2249 There it is 

https://t.co/XklcN5qCLy https://t.co/uOeijQHTp5
2274 @RonFilipkowski No Tesla for me now!!!
2378 @MadeinSaudi @kaec_saudi @mhelal2 

not Tesla 😂🤣😂🤣
2436 @Oculustrade Tesla won’t go down from now on
3441 @Cyberifyy @Rockaah14 Because that’s what tesla is
4775 @resteasyflow Is that a Tesla?
5567 Tesla down 7% 🥶
5585 @GerberKawasaki @PeteButtigieg Tesla about to go below 700. https://t.co/wf5nLsR2JF
5629 @Tesla I have tesla car but no....
7152 S C A M 🤐#tesla https://t.co/

In [22]:
print(len(data_lemmatized), len(lem_noNone), len(tester))

237659 237000 237000


In [23]:
lem_noNone = [i.split() for i in lem_noNone]

In [24]:
id2word = corpora.Dictionary(lem_noNone)
print(len(id2word)) 

66590


In [25]:
# This can be Tuned --- and probably chould be

id2word.filter_extremes(no_below=6, no_above=.90)
print(len(id2word))

18507


In [26]:
# Another potential preprocessing step 
# Could remove tweets that are fewer than n number of words 

counter = 0
for i in lem_noNone:
    if len(i) <= 3:
        counter += 1
counter

32702

In [27]:
corpus = [id2word.doc2bow(d) for d in lem_noNone]

In [42]:
# Instantiating a Base LDA model
base_model = LdaMulticore(corpus=corpus,
                          num_topics=5,
                          id2word=id2word,
                          workers=7,
                          passes=5,
                          chunksize=2500,
                          random_state = 42)
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]
topics = [' '.join(t[0:10]) for t in words]

for id, t in enumerate(topics): 
    print(f"------ Topic {id + 1} ------")
    print(t, end="\n\n")

------ Topic 1 ------
model first spot time think teslaradar day say know standard

------ Topic 2 ------
buy musk elon car people electric make want think say

------ Topic 3 ------
car electric make ev company battery year get vehicle market

------ Topic 4 ------
stock buy get go twitter price share market see back

------ Topic 5 ------
fire door car people driver crash drive use get autopilot



In [43]:
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=lem_noNone, 
                                   dictionary=id2word, coherence='c_v')

coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base, '\n\nPerplexity: ', base_perplexity)


Coherence Score:  0.33998669503645845 

Perplexity:  -7.642658044311896


In [40]:
"""
chunk size = 4000
Coherence Score:  0.3277361656874404 
Perplexity:  -7.656176410369252



chunk size = 1000
Coherence Score:  0.36408864500672655 
Perplexity:  -7.648354363180408"""

'\nchunk size = 1000\nCoherence Score:  0.36408864500672655 \nPerplexity:  -7.648354363180408'

In [41]:
pyLDAvis.enable_notebook()
gensimvis.prepare(base_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.043024  0.031502       1        1  29.594909
1      0.088135 -0.110200       2        1  23.999476
3      0.032734 -0.103158       3        1  19.352818
4      0.069061  0.179123       4        1  17.264735
0     -0.232954  0.002733       5        1   9.788061, topic_info=        Term          Freq         Total Category  logprob  loglift
162    model  11888.000000  11888.000000  Default  30.0000  30.0000
156     elon  17298.000000  17298.000000  Default  29.0000  29.0000
151    stock  24140.000000  24140.000000  Default  28.0000  28.0000
128     musk  19014.000000  19014.000000  Default  27.0000  27.0000
144      buy  26776.000000  26776.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
127   market   1233.367149   9215.511780   Topic5  -5.2639   0.3129
421  someone    908.371327   3200.536320   Topic5  -5.5697   1.0646
151    stock   1258.042049  24140.654290   Topic5  -5.2441  -0.6303
306     good    944.697435  10847.150277   Topic5  -5.5305  -0.1168
94       new    913.154553   8170.111762   Topic5  -5.5645   0.1327

[472 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1609      1  0.011131  accident
1609      2  0.004281  accident
1609      3  0.007706  accident
1609      4  0.974361  accident
1609      5  0.002569  accident
...     ...       ...       ...
355       1  0.507355      year
355       2  0.128376      year
355       3  0.158348      year
355       4  0.045131      year
355       5  0.160860      year

[1289 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 5, 1])

#### ============================================================================

# Tuning topics and pther things .....

### Here
## We 
# Go

In [63]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=1500,
                                           passes=8,
                                           alpha=a,
                                           eta=b, 
                                           workers = 7)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=lem_noNone, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [76]:
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 3
max_topics = 9
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [#gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.5)), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['50% Corpus', '75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    results_df = pd.DataFrame(model_results)
    pbar.close()


  1%|█▍                                                                                                                                                          | 3/336 [31:03:59<3448:23:05, 37279.84s/it]

 11%|██████████████████▌                                                                                                                                                 | 38/336 [38:27<5:11:38, 62.75s/it]


 23%|████████████████████████████████████▋                                                                                                                           | 77/336 [3:05:37<20:17:40, 282.09s/it]


 35%|██████████████████████████████████████████████████████▉                                                                                                        | 116/336 [5:45:36<12:43:46, 208.30s/it]


 46%|█████████████████████████████████████████████████████████████████████████▎                                                                                     | 155/336 [8:39:49<12:53:32, 256.42s/it]


 58%|███████████████████████████████████████████████████████████████████████████████████████████▊                                                                   | 194/336 [11:27:35<9:45:54, 247.57s/it]


 69%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                 | 233/336 [12:45:07<2:08:49, 75.04s/it]


 81%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                              | 272/336 [13:38:40<1:18:47, 73.87s/it]


 93%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉            | 311/336 [14:28:07<32:46, 78.68s/it]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 336/336 [14:58:14<00:00, 82.49s/it]
337it [14:59:40, 83.50s/it]                                                                                                                                                                                 
338it [15:01:08, 84.74s/it]
339it [15:02:33, 85.03s/it]
340it [15:03:56, 84.44s/it]
341it [15:05:04, 79.33s/it]
342it [15:06:41, 84.86s/it]
343it [15:08:26, 90.72s/it]
344it [15:10:10, 94.72s/it]
345it [15:11:36, 92.26s/it]
346it [15:12:52, 87.39s/it]
347it [15:14:05, 82.84s/it]
348it [15:15:22, 81.12s/it]
349it [15:16:46, 81.93s/it]
350it [15:17:59, 79.41s/it]
351it [15:19:21, 80.06s/it]
352it [15:20:41, 80.09s/it]
353it [15:22:04, 80.87s/it]
354it [15:23:33, 83.39s/it]
355it [15:24:53, 82.34s/it]
356it [15:26:12, 81.51s/it]
357it [15:27:36, 82.02s/it]
358it [15:29:01, 82.97s/it]
35

426it [17:23:44, 107.10s/it]
427it [17:25:32, 107.26s/it]
428it [17:27:19, 107.35s/it]
429it [17:29:25, 112.96s/it]
430it [17:31:16, 112.25s/it]
431it [17:32:43, 104.72s/it]
432it [17:34:11, 99.76s/it] 
433it [17:36:00, 102.33s/it]
434it [17:37:42, 102.48s/it]
435it [17:39:24, 102.33s/it]
436it [17:40:45, 95.94s/it] 
437it [17:42:05, 91.09s/it]
438it [17:43:28, 88.72s/it]
439it [17:44:53, 87.47s/it]
440it [17:50:21, 159.65s/it]
441it [18:18:10, 612.32s/it]
442it [18:36:25, 757.37s/it]
443it [19:00:03, 955.53s/it]
444it [19:10:02, 848.54s/it]
445it [19:12:16, 634.23s/it]
446it [19:14:04, 476.18s/it]
447it [19:15:41, 362.58s/it]
448it [19:17:15, 281.78s/it]
449it [19:18:47, 225.13s/it]
450it [19:20:18, 184.83s/it]
451it [19:21:50, 156.81s/it]
452it [19:23:23, 137.69s/it]
453it [19:24:54, 123.74s/it]
454it [19:26:26, 114.28s/it]
455it [19:27:56, 106.82s/it]
456it [19:29:37, 105.33s/it]
457it [19:31:20, 104.46s/it]
458it [19:33:05, 104.56s/it]
459it [19:34:50, 104.81s/it]
460it [19:36:31, 

In [77]:
results_df

,Validation_Set,Topics,Alpha,Beta,Coherence
0,50% Corpus,3,0.01,0.01,0.319492
1,50% Corpus,3,0.01,0.31,0.319296
2,50% Corpus,3,0.01,0.61,0.304459
3,50% Corpus,3,0.01,0.91,0.297005
4,50% Corpus,3,0.01,symmetric,0.319296
...,...,...,...,...,...
535,100% Corpus,8,asymmetric,0.01,0.384329
536,100% Corpus,8,asymmetric,0.31,0.390322
537,100% Corpus,8,asymmetric,0.61,0.342518
538,100% Corpus,8,asymmetric,0.91,0.350760


In [85]:
results_df.describe()

,Topics,Coherence
count,540.000000,540.000000
mean,5.500000,0.363041
std,1.709409,0.030667
min,3.000000,0.289269
25%,4.000000,0.341786
50%,5.500000,0.359458
75%,7.000000,0.389823
max,8.000000,0.444916


In [90]:
results_df[results_df['Coherence'] >= 0.41]

,Validation_Set,Topics,Alpha,Beta,Coherence
150,50% Corpus,8,0.01,0.01,0.418522
170,50% Corpus,8,symmetric,0.01,0.419856
175,50% Corpus,8,asymmetric,0.01,0.444916
178,50% Corpus,8,asymmetric,0.91,0.421200
337,75% Corpus,8,0.31,0.61,0.410594
340,75% Corpus,8,0.61,0.01,0.412862
343,75% Corpus,8,0.61,0.91,0.412157
347,75% Corpus,8,0.91,0.61,0.411244
348,75% Corpus,8,0.91,0.91,0.412627
355,75% Corpus,8,asymmetric,0.01,0.414050


In [95]:
# Instantiating a Base LDA model
newer_model = LdaMulticore(corpus=corpus,
                          num_topics=6,
                          id2word=id2word,
                          workers=7,
                          passes=8,
                          alpha = 0.91 , 
                          eta = 0.61,
                          chunksize=1500,
                          random_state = 100)
newer_words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]
newer_topics = [' '.join(t[0:10]) for t in words]

for id, t in enumerate(newer_topics): 
    print(f"------ Topic {id + 1} ------")
    print(t, end="\n\n")

------ Topic 1 ------
model first spot time think teslaradar day say know standard

------ Topic 2 ------
buy musk elon car people electric make want think say

------ Topic 3 ------
car electric make ev company battery year get vehicle market

------ Topic 4 ------
stock buy get go twitter price share market see back

------ Topic 5 ------
fire door car people driver crash drive use get autopilot



In [96]:
pyLDAvis.enable_notebook()
gensimvis.prepare(newer_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.136688 -0.287719       1        1  18.153902
0     -0.147997  0.090072       2        1  17.866807
2      0.275987 -0.041683       3        1  17.199423
3      0.078218  0.008445       4        1  15.930614
1     -0.101687  0.168697       5        1  15.810154
4      0.032167  0.062188       6        1  15.039100, topic_info=         Term          Freq         Total Category  logprob  loglift
151     stock  24099.000000  24099.000000  Default  30.0000  30.0000
144       buy  23438.000000  23438.000000  Default  29.0000  29.0000
128      musk  17530.000000  17530.000000  Default  28.0000  28.0000
119  electric  20181.000000  20181.000000  Default  27.0000  27.0000
117       car  42188.000000  42188.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
117       car   5375.444979  42188.417822   Topic6  -4.2213  -0.1658
813      life   1541.179008   2078.171732   Topic6  -5.4706   1.5956
72      think   1961.905411  13441.165657   Topic6  -5.2292  -0.0299
73       time   1696.350571  10664.675329   Topic6  -5.3747   0.0561
100      know   1638.787290  11591.810617   Topic6  -5.4092  -0.0618

[419 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1609      1  0.000798  accident
1609      2  0.000798  accident
1609      3  0.000798  accident
1609      4  0.000798  accident
1609      5  0.001596  accident
...     ...       ...       ...
355       2  0.002524      year
355       3  0.969172      year
355       4  0.001218      year
355       5  0.001827      year
355       6  0.012009      year

[2166 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 3, 4, 2, 5])

In [97]:
newer_perplexity = newer_model.log_perplexity(corpus)

# Compute Coherence Score
newer_coherence_model = CoherenceModel(model= newer_model, texts=lem_noNone, 
                                   dictionary=id2word, coherence='c_v')

new_coherence_lda_model_base = newer_coherence_model.get_coherence()
print('\nCoherence Score: ', new_coherence_lda_model_base, '\nPerplexity: ', newer_perplexity)


Coherence Score:  0.4204086107920027 
Perplexity:  -7.653450394242961


In [69]:
corpus[142341]

[(46, 1),
 (147, 1),
 (558, 1),
 (705, 1),
 (1621, 1),
 (2214, 1),
 (2239, 1),
 (3565, 1),
 (18167, 1)]

In [ ]:
"""
BASE PREPROCESSIG ON BASE LDA MODEL

without tesla and single letters 
Coherence Score:  0.292141780223799 
Perplexity:  -7.595869898279568


With 'Tesla' and single letters
Coherence Score:  0.35428295570283674 
Perplexity:  -7.273198516919201"""

# Grid Search is below ==================================

# ====================================================================

#### Attempting a GridSearch on the LDA model

In [92]:
df2 = df.drop('tokens')

In [94]:
df2

0         solar grid run computer radio fridge entire ya...
1                boycott teslastock elonsproblems vote blue
2         never survive carbon offset implement democrat...
3                            funny fanbase take profit sell
4         right nonsense position find sort hard believe...
                                ...                        
237654    yes beg borrow steal invest world valuable com...
237655    owh rethink thesis always think regulation bui...
237656    overused example elon musk alienate almost yea...
237657    found spacex successful also arguably innovati...
237658               want ev table musk show world year old
Name: tweets, Length: 237659, dtype: object

In [91]:
vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(df.drop('tokens'))

In [ ]:
# CV Param
search_params = {'n_components': [5, 10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9], 'batch_size':[128, 256]}

# Init Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, cv = 5, param_grid=search_params, n_jobs = -1)

# Grid Search
model.fit(data_vectorized)

In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

In [ ]:
best_lda_model

In [397]:
id2word.keys()

18507

In [ ]:
GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128, 
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1, 
                                                 learning_decay=0.7, 
                                                 learning_method=None,
                                                 learning_offset=10.0, 
                                                 max_doc_update_iter=100, 
                                                 max_iter=10,
                                                 mean_change_tol=0.001, 
                                                 n_components=10, 
                                                 n_jobs=1,
                                                 perp_tol=0.1, 
                                                 random_state=None,
                                                 topic_word_prior=None, 
                                                 total_samples=1000000.0, 
                                                 verbose=0),
             n_jobs=-2,
             param_grid={'n_topics': [10, 15, 20, 30], 
                         'learning_decay': [0.5, 0.7, 0.9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
             scoring=None, verbose=0)